# Errors and Exceptions

There are two distinguishable kinds of errors: *syntax errors* and *exceptions*.
- Syntax errors, also known as parsing errors, are the most common.
- Exceptions are errors caused by statement or expression syntactically corrects.
- Exceptions are not unconditionally fatal.

[Exceptions in Python documentation](https://docs.python.org/3/library/exceptions.html)

In [1]:
10 * (1/0)

ZeroDivisionError: division by zero

In [2]:
4 + spam*3

NameError: name 'spam' is not defined

In [3]:
'2' + 2

TypeError: must be str, not int

# Handling Exceptions

- In example below, the user can interrupt the program with `Control-C` or the `stop` button in Jupyter Notebook.
- Note that a user-generated interruption is signalled by raising the **KeyboardInterrupt** exception.


In [7]:
while True:
   try:
     x = int(input("Please enter a number: "))
     break
   except ValueError:
     print("Oops!  That was no valid number.  Try again...")

Please enter a number: p
Oops!  That was no valid number.  Try again...
Please enter a number: o
Oops!  That was no valid number.  Try again...
Please enter a number: 89.5
Oops!  That was no valid number.  Try again...
Please enter a number: 8


- A try statement may have more than one except clause
- The optional `else` clause must follow all except clauses.

In [17]:
import sys

def process_file(file):
    try:
        i = int(open(file).readline().strip()) # Read the first line of f and convert to int
        print(i)
        assert i < 0 # check if i is negative
    except OSError as err:
        print(f"OS error: {err}")
    except ValueError:
        print("Could not convert data to an integer.")
    except:
        print("Unexpected error:", sys.exc_info()[0])

# Create the file workfile.txt
with open('workfile.txt','w') as f:
    f.write("foo")
    f.write("bar")

In [18]:
process_file('workfile.txt')

OS error: [Errno 13] Permission denied: 'workfile.txt'


In [19]:
# Change permission of the file, workfile.txt cannot be read
!chmod u-r workfile.txt

In [20]:
process_file('workfile.txt')

OS error: [Errno 13] Permission denied: 'workfile.txt'


In [23]:
# Let's delete the file workfile.txt
!rm -f workfile.txt

In [25]:
process_file('workfile.txt')

OS error: [Errno 2] No such file or directory: 'workfile.txt'


In [26]:
# Insert the value 1 at the top of workfile.txt
!echo "1" > workfile.txt
%cat workfile.txt

1


In [27]:
process_file('workfile.txt')

1
Unexpected error: <class 'AssertionError'>


# Raising Exceptions

The raise statement allows the programmer to force a specified exception to occur.


In [6]:
raise NameError('HiThere')

NameError: HiThere

# Defining Clean-up Actions

- The try statement has an optional clause which is intended to define clean-up actions that must be executed under all circumstances.

- A finally clause is always executed before leaving the try statement

In [58]:
try:
     raise KeyboardInterrupt
finally:
     print('Goodbye, world!')

Goodbye, world!


KeyboardInterrupt: 

### Exercise

- Write a function `check_date` that takes a string "DD/MM/YYYY" as argument and
returns `True` if the date is valid.
- Use it with a `try ... except` statement to help the user to enter a valid date.
- raise ValueError "Not a valid date"
- Hints: 
  * Use string method `split`
  * Year y is a leap year if y%400==0 or (y%4==0 and y%100!=0)

<button data-toggle="collapse" data-target="#date" class='btn btn-primary'>Solution</button>
<div id="date" class="collapse">
```python
def check_date(date):
    d, m, y = date.split("/")
    d, m, y = int(d), int(m), int(y)

    months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    if 12 < m or m < 1:  # m in [1,12]
        return False

    if y%400==0 or (y%4==0 and y%100!=0): # leap year
        months[1] = 29

    if months[m-1] < d or d < 1:
        return False
    
    return True

while True:
    try:
        if check_date(input("Please enter a date (DD/MM/YYYY) : ")):
            print("OK")
            break
        else:
            raise ValueError()

    except ValueError:
        print("Oops!  That was no valid date.  Try again...")
```

### Wordcount Exercise
[WordCount](https://hadoop.apache.org/docs/current/hadoop-mapreduce-client/hadoop-mapreduce-client-core/MapReduceTutorial.html#Example:_WordCount_v1.0) is a simple application that counts the number of occurrences of each word in a given input set. This computation can be splitted into two steps: a map operation to each word in our input text in order to compute a set of intermediate key/value pairs, and then a reduce operation to all the values that shared the same key.

1. Write the function `mapper` with multilines string as input and breaks it into words. It returns a list of key/value pair (word, 1). 
2. Write the funtion `reducer` to read the results of `mapper` and sum the occurrences of each word to a final count, and then output the results as a dictionary of {word:occurences}. 

Hints : 
 - `str.strip()` removes end line character
 - `str.split()` split string into a list of words
 - Use the `KeyError` exception to fill in the dictionary.
 
Here an example of wordcount using unix commands.

In [57]:
!echo "Ce ne sont pas tes erreurs Ce ne sont pas tes triomphes Ce ne sont pas tes années" |  fmt -1 | sort | uniq -c

   3 Ce
   1 années
   1 erreurs
   3 ne
   3 pas
   3 sont
   3 tes
   1 triomphes


<button data-toggle="collapse" data-target="#mapper" class='btn btn-primary'>Solution mapper</button>
<div id="mapper" class="collapse">
```python
def mapper ( text ):
    """ Count every occurence of word in text
    return a sorted list of tuples (word,1) """
    result = []
    for word in text.strip().split():
        result.append((word,1))
    return sorted(result)

text = """
        Ce ne sont pas tes erreurs
        Ce ne sont pas tes triomphes
        Ce ne sont pas tes années
       """
mapper(text)
```

<button data-toggle="collapse" data-target="#reducer" class='btn btn-primary'>Solution reducer without Exception</button>
<div id="reducer" class="collapse">
```python       
def reducer ( mapped_values):
    """ Count the number of occurences of a word in a sorted list
    of tuples (word,1), and return result in a dictionary """
    
    current_word = None
    result = {}
    for word, k in mapped_values:
        if current_word is None:  
            current_word = word
            result[word] = 0  # Add the first word in result

        # this if only works because input is sorted 
        if current_word == word:
            result[word] += k
        else: # When the word is not already present in result
            current_word = word 
            result[word] = k

    return result
    
reducer(mapper(text))
```

<button data-toggle="collapse" data-target="#keyerror" class='btn btn-primary'>Solution reducer with Exception</button>
<div id="keyerror" class="collapse">
```python
def reducer(words):
    """ Read the sorted list of (words,1) tuples and fill a dictionary with every word 
    as keys and number of occurences as values"""
    result = {}
    for w, k in words:
        try:
            result[w] +=k
        except KeyError:
            result[w] = k 
    return result
```